In [100]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import ltrim,rtrim,trim,col
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("MyFirstCSV").getOrCreate()


In [101]:
oij_df = spark.read.csv( path="OIJ.csv", sep=",", header=True,quote='"',inferSchema=True,)
inec_df = spark.read.csv( path="INEC.csv", sep=";", header=True,quote='"',inferSchema=True,)

In [ ]:
def remove_spaces(df):
    for col in df.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito' or col == 'Provincia, cantón y distrito':
            df = df.withColumn(col, trim(col))
    return df


def to_lower_case(df):
    for col in df.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito' or col == 'Provincia, cantón y distrito':
            df = df.withColumn(col, lower(col))
    return df


oij_df = remove_spaces(oij_df)
inec_df = remove_spaces(inec_df)

oij_df = to_lower_case(oij_df)
inec_df = to_lower_case(inec_df)

oij_df.show(5)
inec_df.show(5)

In [103]:
def find_non_matches(df1,df2):
    non_matches = []
    for col in df1.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito' or col == 'Provincia, cantón y distrito':
            non_matches.append(df1.join(df2, df1[col] != df2[col], 'leftanti').select(col).distinct().collect())
    return non_matches

non_matches_oij = find_non_matches(oij_df,inec_df)

for i in non_matches_oij:
    print(i)

AnalysisException: Cannot resolve column name "Provincia" among (Provincia, cantón y distrito, Población de 15 años y más, Tasa neta de participación, Tasa de ocupación, Tasa de desempleo abierto, Porcentaje de población económicamente inactiva, Relación de dependencia económica, Porcentaje de población ocupada  Sector Primario, Porcentaje de población ocupada  Sector Secundario, Porcentaje de población ocupada  Sector Terciario)